In [1]:
import pandas as pd
from utide import solve, reconstruct
import matplotlib.pyplot as plt
import numpy as np

/home/gustavo/.cache/pypoetry/virtualenvs/pcce-tides-tPVxqpqz-py3.8/lib/python3.8/site-packages/utide/harmonics.py:16: RuntimeWarning: invalid value encountered in cast
  nshallow = np.ma.masked_invalid(const.nshallow).astype(int)
/home/gustavo/.cache/pypoetry/virtualenvs/pcce-tides-tPVxqpqz-py3.8/lib/python3.8/site-packages/utide/harmonics.py:17: RuntimeWarning: invalid value encountered in cast
  ishallow = np.ma.masked_invalid(const.ishallow).astype(int) - 1


In [2]:
#import utide

In [3]:
#dfu = pd.read_csv('../data/ul-boia-completo.csv', delimiter=';')
#dfv = pd.read_csv('../data/ul-boia-completo.csv', delimiter=';')

In [ ]:
#u = dfu.iloc[0:3000, 1]
#v = dfv.iloc[0:3000, 1]

In [ ]:
#dfu.head(3)

In [7]:
#time = dfu.time.iloc[:3000]
#time = pd.to_datetime(time)

In [ ]:
#time

In [ ]:
#u.isna().sum(), v.isna().sum()

In [ ]:
#u.interpolate(method='linear', limit=12, inplace=True)
#v.interpolate(method='linear', limit=12, inplace=True)

### Creataing a white noise

In [ ]:
#n = len(time) # comprimento
#amp = 0.001 # amplitude
#phase = np.pi/4 # fase em radianos
#t = 500 # horas
#t2 = 800 #
#white_noise = [amp + np.sin((i*np.pi/2)/t) for i in range(n)]
#white_noise2 = [amp + np.sin((i*np.pi/2)/t2 + phase) for i in range(n)]

#white_noise = np.array(white_noise)
#white_noise2 = np.array(white_noise2)

In [ ]:
#plt.plot(white_noise)
#plt.plot(white_noise2)

In [3]:
def add_noise(vector, noise) -> np.asarray:
    from numpy import isnan
    vector[isnan(vector)] = 0
    vector = vector + noise
    return vector

In [37]:
def T_tide(time:np.datetime64, c:np.asarray):
    global coef
    coef = solve(time, c,
                 lat=-3,
                 nodal=False,
                 trend=False,
                 method='robust',
                 conf_int='linear',
                 Rayleigh_min=0.95,)
    
    print('Name\tAmp\tPhase\tFreq\tPE\tSNR\t')
    for name, amp, phase, frq ,pe, snr in zip(coef.name, coef.A, coef.g, coef.aux.frq ,coef.PE, coef.SNR):
        print(' {}\t{:.2f}\t{:.1f}\t{:.4f}\t{:.2f}\t {:.2f}\t'.format(name, amp, phase, frq, pe, snr))

In [38]:
dfu = pd.read_csv('../data/ul-boia-completo.csv', delimiter=';')
dfv = pd.read_csv('../data/ul-boia-completo.csv', delimiter=';')


time = dfu.time.iloc[:3000]
time = pd.to_datetime(time)

# ruido branco
n = len(time) # comprimento
amp = 0.001 # amplitude
phase = np.pi/4 # fase em radianos
t = 1500 # T horas
t2 = 1800 # T horas

#add ruido no dado
white_noise = [amp + np.sin((i*np.pi/2)/t) for i in range(n)]
white_noise2 = [amp + np.sin((i*np.pi/2)/t2 + phase) for i in range(n)]
white_noise = np.array(white_noise)
white_noise2 = np.array(white_noise2)

# peformando a análise harmonica
for depth in [1, 10, 19]:    
    
    uvec = dfu.iloc[:3000, depth]
    vvec = dfv.iloc[:3000, depth]
    uvec = uvec-np.nanmean(uvec)
    vvec = vvec-np.nanmean(vvec)
    
    uvec.interpolate(method='linear', limit=12, inplace=True)
    vvec.interpolate(method='linear', limit=12, inplace=True)
    
    uvec, vvec = add_noise(uvec, noise=white_noise), add_noise(vvec, noise=white_noise2)
    complex_vec = uvec + vvec*1j
    
    T_tide(time, complex_vec)

solve: matrix prep ... solution ... done.
Name	Amp	Phase	Freq	PE	SNR	
 M2	7.52	317.6	0.0805	78.33	 3588.95	
 S2	2.26	330.2	0.0833	7.07	 323.92	
 MM	2.19	198.2	0.0015	6.64	 43.58	
 N2	1.41	301.3	0.0790	2.74	 125.59	
 K1	1.01	263.2	0.0418	1.41	 53.70	
 MSF	0.67	202.6	0.0028	0.62	 4.05	
 NO1	0.62	330.9	0.0403	0.53	 20.28	
 MK3	0.44	224.4	0.1223	0.27	 33.11	
 2SK5	0.43	258.3	0.2084	0.26	 73.13	
 MU2	0.41	13.9	0.0777	0.23	 10.60	
 MN4	0.38	220.7	0.1595	0.20	 42.98	
 M4	0.32	179.1	0.1610	0.14	 31.80	
 ETA2	0.31	79.6	0.0851	0.14	 6.22	
 S4	0.31	43.2	0.1667	0.13	 29.40	
 2Q1	0.30	48.5	0.0357	0.13	 4.78	
 SN4	0.30	190.7	0.1623	0.12	 27.45	
 2MN6	0.28	47.7	0.2400	0.11	 32.50	
 M3	0.28	315.0	0.1208	0.11	 13.13	
 Q1	0.27	200.4	0.0372	0.10	 3.97	
 MO3	0.27	149.9	0.1192	0.10	 12.72	
 OO1	0.27	262.1	0.0448	0.10	 3.81	
 O1	0.26	82.4	0.0387	0.09	 3.58	
 ALP1	0.24	280.5	0.0344	0.08	 3.09	
 MS4	0.23	130.8	0.1638	0.08	 16.79	
 UPS1	0.23	240.1	0.0463	0.07	 2.75	
 J1	0.20	49.1	0.0433	0.06	 2.17	
 M6	0.18	37

#from utide import solve
coef = solve(time, c,
             lat=-3,
             nodal=False,
             trend=False,
             method='robust',
             conf_int='linear',
             Rayleigh_min=0.95,)

In [36]:
coef.keys()

dict_keys(['name', 'aux', 'nR', 'nNR', 'nI', 'rf', 'weights', 'A', 'g', 'mean', 'g_ci', 'A_ci', 'diagn', 'PE', 'SNR'])